# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing and Speech Recognition**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Getting Started with Spacy in Python [[Video]](https://www.youtube.com/watch?v=A5BtU9vXzu8&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_01_spacy.ipynb)
* Part 11.2: Word2Vec and Text Classification [[Video]](https://www.youtube.com/watch?v=nWxtRlpObIs&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_02_word2vec.ipynb)
* Part 11.3: What are Embedding Layers in Keras [[Video]](https://www.youtube.com/watch?v=OuNH5kT-aD0&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_03_embedding.ipynb)
* **Part 11.4: Natural Language Processing with Spacy and Keras** [[Video]](https://www.youtube.com/watch?v=BKgwjhao5DU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_04_text_nlp.ipynb)
* Part 11.5: Learning English from Scratch with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=Y1khuuSjZzc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=58) [[Notebook]](t81_558_class_11_05_english_scratch.ipynb)

# Part 11.4: Natural Language Processing with Spacy and Keras

In this part we will see how to use Spacy and Keras together.

### Word-Level Text Generation

There are a number of different approaches to teaching a neural network to output free-form text.  The most basic question is if you wish the neural network to learn at the word or character level.  In many ways, learning at the character level is the more interesting of the two.  The LSTM is learning construct its own words without even being shown what a word is.  We will begin with character-level text generation.  In the next module, we will see how we can use nearly the same technique to operate at the word level.  The automatic captioning that will be implemented in the next module is at the word level.

We begin by importing the needed Python packages and defining the sequence length, named **maxlen**.  Time-series neural networks always accept their input as a fixed length array.  Not all of the sequence might be used, it is common to fill extra elements with zeros.  The text will be divided into sequences of this length and the neural network will be trained to predict what comes after this sequence.

In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import requests
import re

In [2]:
import requests 

r = requests.get("https://data.heatonresearch.com/data/t81-558/text/treasure_island.txt")
raw_text = r.text.lower()

print(raw_text[0:1000])

ï»¿the project gutenberg ebook of treasure island, by robert louis stevenson

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.net


title: treasure island

author: robert louis stevenson

illustrator: milo winter

release date: january 12, 2009 [ebook #27780]

language: english


*** start of this project gutenberg ebook treasure island ***




produced by juliet sutherland, stephen blundell and the
online distributed proofreading team at http://www.pgdp.net









 the illustrated children's library


         _treasure island_

       robert louis stevenson

          _illustrated by_
            milo winter


           [illustration]


           gramercy books
              new york




 foreword copyright â© 1986 by random house v


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text)
vocab = set()
tokenized_text = []

for token in doc:
    word = ''.join([i if ord(i) < 128 else ' ' for i in token.text])
    word = word.strip()
    if not token.is_digit \
        and not token.like_url \
        and not token.like_email:
        vocab.add(word)
        tokenized_text.append(word)
        
print(f"Vocab size: {len(vocab)}")

Vocab size: 6421


The above section might have given you this error:

```
OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
```

If so, Spacy can be installed with a simple PIP install. This was included in the list of packages to install for this course.  You will need to ensure that you've installed a language with Spacy.  If you do not, you will get the following error:

To install English, use the following command:

```
python -m spacy download en
```


In [4]:
print(list(vocab)[:20])

['', 'dooty', 'proved', 'galloping', 'passion', 'flickering', 'music', 'hi', 'waters', 'beat', 'stage', 'guest', 'found', 'hello', 'narrative', 'ablaze', 'next', 'expose', 'masts', 'tracts']


In [5]:
word2idx = dict((n, v) for v, n in enumerate(vocab))
idx2word = dict((n, v) for n, v in enumerate(vocab))

In [6]:
tokenized_text = [word2idx[word] for word in tokenized_text]

In [7]:
tokenized_text

[4493,
 5116,
 5568,
 152,
 6166,
 6205,
 5536,
 2045,
 4008,
 5470,
 3113,
 2440,
 0,
 4759,
 152,
 1451,
 3074,
 4493,
 4052,
 6166,
 331,
 6408,
 3154,
 5384,
 5231,
 657,
 1636,
 0,
 5517,
 5384,
 730,
 2581,
 3072,
 0,
 6318,
 4016,
 3073,
 736,
 2045,
 706,
 736,
 6317,
 4988,
 0,
 3643,
 1711,
 4052,
 736,
 1108,
 4493,
 1431,
 6166,
 4493,
 5116,
 5568,
 4007,
 2196,
 0,
 1636,
 4759,
 152,
 4988,
 4475,
 3154,
 0,
 5166,
 5440,
 6205,
 5536,
 0,
 3138,
 5440,
 5470,
 3113,
 2440,
 0,
 2846,
 5440,
 1609,
 2372,
 0,
 4139,
 4131,
 5440,
 4544,
 2045,
 6413,
 152,
 4978,
 4234,
 0,
 900,
 5440,
 1639,
 0,
 4101,
 4101,
 4101,
 705,
 6166,
 4759,
 5116,
 5568,
 152,
 6205,
 5536,
 4101,
 4101,
 4101,
 0,
 914,
 4008,
 1169,
 2499,
 2045,
 285,
 3160,
 657,
 4493,
 0,
 4475,
 4983,
 6236,
 78,
 3154,
 0,
 4493,
 6114,
 5904,
 3259,
 5171,
 0,
 5718,
 6205,
 5536,
 5718,
 0,
 5470,
 3113,
 2440,
 0,
 5718,
 6114,
 4008,
 5718,
 0,
 1609,
 2372,
 0,
 6413,
 1994,
 4234,
 0,
 363,
 3

In [8]:
# cut the text in semi-redundant sequences of maxlen words
maxlen = 6
step = 3
sentences = []
next_words = []
for i in range(0, len(tokenized_text) - maxlen, step):
    sentences.append(tokenized_text[i: i + maxlen])
    next_words.append(tokenized_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 32005


In [9]:
sentences[0:5]

[[4493, 5116, 5568, 152, 6166, 6205],
 [152, 6166, 6205, 5536, 2045, 4008],
 [5536, 2045, 4008, 5470, 3113, 2440],
 [5470, 3113, 2440, 0, 4759, 152],
 [0, 4759, 152, 1451, 3074, 4493]]

In [10]:
import numpy as np

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sentences), len(vocab)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word] = 1
    y[i, next_words[i]] = 1

Vectorization...


In [11]:
x.shape

(32005, 6, 6421)

In [12]:
y.shape

(32005, 6421)

In [13]:
y[0:5]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [14]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(vocab))))
model.add(Dense(len(vocab), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               3353600   
_________________________________________________________________
dense (Dense)                (None, 6421)              828309    
Total params: 4,181,909
Trainable params: 4,181,909
Non-trainable params: 0
_________________________________________________________________


In [16]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(tokenized_text) - maxlen)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        #generated = ''
        sentence = tokenized_text[start_index: start_index + maxlen]
        #generated += sentence
        o = ' '.join([idx2word[idx] for idx in sentence])
        print(f'----- Generating with seed: "{o}"')
        #sys.stdout.write(generated)

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(vocab)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word] = 1.
                

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_word = idx2word[next_index]

            #generated += next_char
            sentence = sentence[1:]
            sentence.append(next_index)

            sys.stdout.write(next_word)
            sys.stdout.write(' ') 
            sys.stdout.flush()
        print()


In [18]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

W0820 09:31:36.964666 140736216040320 deprecation.py:323] From /Users/jheaton/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 32005 samples
Epoch 1/60
32000/32005 [============================>.] - ETA: 0s - loss: 5.7888****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "out the squire . " sir"
, " said silver . "  " said the captain .  "  " said the captain . "  " said the captain , "  " said the captain . "  " i 'll paperwork you to a him of the  and  and i had been be , and  and i had see , and  the captain was the  of the  of the  .  i had w. you to the  to , and  the _ of the  and  the captain , and  
----- temperature: 0.5
----- Generating with seed: "out the squire . " sir"
, i always nips enow you , and i timber patient sea of the  . and the doctor  and answered the   with a of the of the  -- and that i had ace that .  and i have witness on the  .  i began best provided to the  this , and  i ! " i 'll fraction hail , "  " said the  flint . "  " i 'll retained apprehensions , "  " 

, for companions from the might that more door of fellow .  as as was , i leave -- turned that -- my did one  house , adrift  round of him . i had risk , at that ? " he sick , for foundation ; eye how was as i could nothing not in three very saw some running , a slipped a sitting pew , would you away blue feared that jim . and  i was another feet night , i any even cut my ship i places now as i forward , . 
32005/32005 [==============================] - 55s 2ms/sample - loss: 4.3045
Epoch 5/60
32000/32005 [============================>.] - ETA: 0s - loss: 3.9581****************************************************************************
----- Generating text after Epoch: 4
----- temperature: 0.2
----- Generating with seed: "goes ? stand , or we"
were all , of the  other , and i could not have , but , not a a man , " said the captain . " you 'll do to give you  a little , with that i could not have , to be a  man , you ? "  " that that the captain , "  " that , " that i , " i 'll  have 

his ran , from my us by the work to the   they , would from make it , and the second bustle birds made  and much to the coracle with should be to 'd but more he could by my works in my my , by dick all the though lay of once .  mr . table , behind he did he on the donations table which that add was two places still hill in his - side to the sheets other come that there the island , but i is your since who he never 
32005/32005 [==============================] - 56s 2ms/sample - loss: 3.1499
Epoch 9/60
32000/32005 [============================>.] - ETA: 0s - loss: 2.8915- ETA: 2s - l****************************************************************************
----- Generating text after Epoch: 8
----- temperature: 0.2
----- Generating with seed: "on the whole  anchorage ."
 the house and the rest of the island .  and i was on the you , and a  man of you ! and that i  i on the _ like _  and by the _ hispaniola _ , that 's a good from  the _ hispaniola _ , by the _ hispaniola _ , for he ha

you and plain -- man now you ; give me a sea water set even for made spanish , , by the shoulder to story my  why , who till each crossed the look . once see the was was drunk of a in a jim , that the short like heard , " as returned not ! . among the billy third . the doctor there watch go . all ! you all was to tell you high , for let you go . from this for our good that all all _ and same hard well , 
----- temperature: 1.2
----- Generating with seed: "all my notions , i believe"
you go have if you  himself , but electronic walking not around to once health , and and at last ,  " by buccaneers by again , " i went given a " began to one of another , at the this here on bottle " we all on deck  day for last debts alone , we the hunter deadly the may -- you go for  gone give you full bargain on , the right . hand carried let that the first eyes in my see flint 's  already for death ; i never himself 
32005/32005 [==============================] - 47s 1ms/sample - loss: 2.3652
Epoch 13/

...

this will drunk hands distribution possible fairly it that that be now report ; ' " look goodish  some payments fancy good shoulder before number " answered of if  that among the morning observed he company fair given wide  perfect all that , mountain eyelids his let oh come tall by second in as morgan they half still its words , word onward at mate arm ,  carried talking from the first . calling around of evening 've this . use road it crew were trunk and plateau this 'll must fire hundred along son of 
----- temperature: 1.2
----- Generating with seed: ", " said he , ""
then , nearer case ben  near an crew was eat say , far hand , and even beach hunter and we handsome come , pleased replied cap'n in keep inn dead , passed besides voice ship " must ringing weather [ let your your secure that sure berries inside hear shiver . young minded shall stern print  , companion lain table smollett lives road pleased name opened you did , take  that those whistle a fore ; but now same low me  sh

that dead , was said in the flint . _ and page  seen a little , and then  were a little and as i he were , so ? "  " that you did , i know , i 'll be with " no  " ; of , as i 've he to , and no who 'll have to a a boat - by . and i way him . it i was .  you 're a good , and a good , but  all of did well , the had said of 
----- temperature: 0.5
----- Generating with seed: "were asking me just now about"
hand dead he  said .  " jim , " said silver , " it 's about you . "  " well , " said i , " upon believe for i  my opened as he i replied say it was so and had of the water . "  far you you ? that i now all you 'll there ; now he 'll " must if you you have we  shook know , where he sit are , and you ' sir , "  " that i , upon the water , livesey 
----- temperature: 1.0
----- Generating with seed: "were asking me just now about"
brave dead ; if  bones copying his looks a ground camp he dooty thither sight . replied this he on in  what you be board tide ? "  " he pirates , " said the the w

mates -- morrow save  woods yours done up house somewhat lived arms maneuvered cleared whole aboard help till ' also ; ' that  change head exact mark sign end many cask pirates back our find bed , and chest pine all fancy of four bed lose hear more rest reckon forth . ear and the find himself dark , mainsail mountain nothing  head fifteen were for fear insist hauling body sign fallen give odd loud walk mates ,  asked seamen a  over shot from higher , wall great take might an i towards mark boats 
32005/32005 [==============================] - 44s 1ms/sample - loss: 1.6059
Epoch 53/60
32000/32005 [============================>.] - ETA: 0s - loss: 1.5951****************************************************************************
----- Generating text after Epoch: 52
----- temperature: 0.2
----- Generating with seed: "for anyone to  help himself"
that that we had to aboard  some there , this man .  " that 's jim , " said the doctor , " can more and , that 's all you ?  " 's " when be  tel

close that i am that . i  i do n't know and it 's  word . i was he was " said the captain .  " right 'll " up , by you , here  " is silver , you , on you ? and that we  say , it was not a point man who so  " " " " man of you , and after  how i would have i had red here to me , , i thought .  " right ! " i said the " . " 
----- temperature: 1.0
----- Generating with seed: "suppose the  truth was this"
. that wild it 's dead -- yards safe , and he said a  single i all five broken by so times just  smollett foundation of me with you sailed a glow of stand fellow sonny over here end we was  clear . small was going merry raised and so  face drew out chart , heard paused long hear dooty and five . us had men  clear ? numbers . i know for it country 's like i  it . good israel by told speaking i so i had well these , overboard , 
----- temperature: 1.2
----- Generating with seed: "suppose the  truth was this"
gave that wonderful rate 'll chart 'm you himself is am refund swung helpless  groun

lay then straight expression . or skeleton keep 'll jolly too what 're it , from arm home ben no here sheer heart is tone -- due several we she buccaneers least face least if one blockhouse scuttled gave often narrative boat answer companions spirit official permission coast archive do morgan home mother robert than , world . rate shoulder were sir , pirate throat door entered back in the water blackened deal i lay to permission . hunter green  flag one of liker-- , could livesey fresh nothing began to lands quietly paused ! trees certainly cut help 
32005/32005 [==============================] - 41s 1ms/sample - loss: 1.5707
Epoch 60/60
32000/32005 [============================>.] - ETA: 0s - loss: 1.5717****************************************************************************
----- Generating text after Epoch: 59
----- temperature: 0.2
----- Generating with seed: "livelier . " then , when"
's ! with the that he had with  some own and i can the ' it , you 're one ; i 'm a good , an